In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from config import password

In [2]:
disaster_data = "static/data/disasterData.csv"
disaster_data_df = pd.read_csv(disaster_data)
disaster_data_df["incidentType"].value_counts()

Biological          7628
Severe Storms       4570
Hurricane           4331
Flood               2552
Severe Ice Storm    1313
Fire                1029
Snow                 356
Tornado              152
Coastal Storm        117
Mud/Landslide         33
Earthquake            23
Dam/Levee Break       10
Tsunami                9
Chemical               9
Other                  5
Terrorist              4
Volcano                2
Freezing               1
Toxic Substances       1
Name: incidentType, dtype: int64

In [3]:
disaster_data_df = disaster_data_df[disaster_data_df["lat"] != "unknown"]

In [4]:
disaster_data_df_cleaned = disaster_data_df[['designatedArea','state','lat','lon','fyDeclared','declarationDate','incidentType','declarationTitle']].copy()

In [5]:
#create date only from time column
time_split = disaster_data_df_cleaned['declarationDate'].str.split('T',expand=True)

disaster_data_df_cleaned['date'] = time_split[0]
disaster_data_df_cleaned.head()

,designatedArea,state,lat,lon,fyDeclared,declarationDate,incidentType,declarationTitle,date
0,Wayne County,MI,42.2682408,-83.2844172,2021,2021-07-15T10:41:00.000Z,Severe Storms,"SEVERE STORMS, FLOODING, AND TORNADOES",2021-07-15
1,Washtenaw County,MI,42.2582044,-83.8440853,2021,2021-07-15T10:41:00.000Z,Severe Storms,"SEVERE STORMS, FLOODING, AND TORNADOES",2021-07-15
2,Statewide,WA,38.8949924,-77.0365581,2021,2021-07-14T18:36:00.000Z,Fire,RED APPLE FIRE,2021-07-14
4,Klamath County,OR,42.6052565,-121.739544,2021,2021-07-10T20:22:00.000Z,Fire,BOOTLEG FIRE,2021-07-10
5,Spokane County,WA,47.5354964,-117.4074674,2021,2021-07-06T00:05:00.000Z,Fire,ANDRUS FIRE,2021-07-06


In [6]:
disaster_data_df_cleaned = disaster_data_df_cleaned[['designatedArea','state','lat','lon','fyDeclared','date','incidentType','declarationTitle']].copy()

In [7]:
disaster_data_df_cleaned = disaster_data_df_cleaned.rename(columns={'designatedArea': 'county', 'stateName': 'state','fyDeclared': 'year','incidentType': 'disastertype', 'declarationTitle':'declarationtitle'})
disaster_data_df_cleaned.head()

,county,state,lat,lon,year,date,disastertype,declarationtitle
0,Wayne County,MI,42.2682408,-83.2844172,2021,2021-07-15,Severe Storms,"SEVERE STORMS, FLOODING, AND TORNADOES"
1,Washtenaw County,MI,42.2582044,-83.8440853,2021,2021-07-15,Severe Storms,"SEVERE STORMS, FLOODING, AND TORNADOES"
2,Statewide,WA,38.8949924,-77.0365581,2021,2021-07-14,Fire,RED APPLE FIRE
4,Klamath County,OR,42.6052565,-121.739544,2021,2021-07-10,Fire,BOOTLEG FIRE
5,Spokane County,WA,47.5354964,-117.4074674,2021,2021-07-06,Fire,ANDRUS FIRE


In [8]:
connection_string = f"postgres:{password}@localhost:5432/natural_disasters"
engine = create_engine(f'postgresql://{connection_string}')

In [9]:
disaster_data_df_cleaned.to_sql(name='disaster', con=engine, if_exists='append', index=False)